### **Respuestas de ejercicios 2**
**Curso:** CC3S2 - NLP/LLMs  
**Archivo base:** `Cargador_datos_NLP.ipynb`  


> **Instrucciones**  
> - Completa los ejercicios **sin pegar soluciones completas** desde otros repositorios.  
> - Usa esta plantilla como **cuaderno de trabajo**, agrega celdas adicionales si lo necesitas.  
> - Captura evidencias (shapes, fragmentos de tensores, tablas breves).  
> - Mantén un estilo claro, comenta lo esencial y **no expongas credenciales**.  


#### **Verificación rápida de entorno**

Ejecuta la celda y anota las versiones relevantes. Si hay warnings importantes, coméntalos.


In [ ]:
# TODO: ejecuta y conserva la salida en el notebook
import sys, platform, torch
print("Python:", sys.version)
print("Plataforma:", platform.platform())
print("PyTorch:", torch.__version__)

# Si tienes torchtext / spacy / transformers instalados, descomenta para registrar versiones
try:
    import torchtext
    print("torchtext:", torchtext.__version__)
except Exception as e:
    print("torchtext: no disponible ->", type(e).__name__)

try:
    import spacy
    print("spacy:", spacy.__version__)
except Exception as e:
    print("spacy: no disponible ->", type(e).__name__)

try:
    import transformers
    print("transformers:", transformers.__version__)
except Exception as e:
    print("transformers: no disponible ->", type(e).__name__)



#### **(Opcional) Semillas de reproducibilidad**
Usa esta celda si planeas comparar resultados entre corridas.


In [ ]:
# TODO: fija semillas si lo requieres (opcional)
import random, numpy as np, torch
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
print("Semillas fijadas:", SEED)


#### **Parte B-Mini-labs con `Cargador_datos_NLP.ipynb`**

> Meta: dominar tokenización, vocabularios, `collate_fn`, *padding*, BOS/EOS y batching.  
> Notación: usa **PyTorch** (y torchtext/spacy si están disponibles en el entorno).


**Ejercicio 8 - Dataset mínimo y DataLoader**

**Objetivo:** crear `CustomDataset` (devuelve cadenas) y `DataLoader`.  
**Pasos sugeridos:**  
1. Define 6-8 oraciones simples.  
2. Implementa `__len__` y `__getitem__`.  
3. Itera sobre 2 lotes y muestra su contenido.  
**Entrega:** pantallazo de lotes.  
**Mini-check:** `shuffle=True` debe cambiar el orden.


In [ ]:
# TODO: implementa un Dataset mínimo y un DataLoader
from typing import List
from torch.utils.data import Dataset, DataLoader

class CustomTextDataset(Dataset):
    def __init__(self, sentences: List[str]):
        self.sentences = sentences
    def __len__(self):
        # TODO: retorna la cantidad de elementos
        return len(self.sentences)
    def __getitem__(self, idx: int):
        # TODO: retorna la cadena en la posición idx
        return self.sentences[idx]

sentences = [
    "Hola desde Lima",
    "El NLP requiere buenos datos",
    "El padding alinea tensores",
    "spacy mejora la tokenización",
    "Multi30k contiene pares de oraciones",
    "HF integra tokenizadores",
    "PyTorch DataLoader crea lotes",
    "Bucketing reduce padding waste"
]

ds = CustomTextDataset(sentences)
dl = DataLoader(ds, batch_size=4, shuffle=True)

for i, batch in enumerate(dl):
    print(f"Lote {i}:", batch)
    if i == 1:
        break

assert len(ds) == len(sentences) 


**Ejercicio 9-Tokenizador + vocabulario**

**Objetivo:** convertir a tensores de índices.  
**Sugerencias:**  
- Usa `get_tokenizer("basic_english")` si no tienes spaCy.  
- Construye el vocabulario con `build_vocab_from_iterator` (incluye tokens especiales).  
- Devuelve `torch.tensor(ids)` desde el Dataset.  
**Entrega:** 2 ejemplos (tensor + lista de tokens).  


In [ ]:
# TODO: tokenizar y construir vocabulario
import torch
torchtext.disable_torchtext_deprecation_warning()
try:
    from torchtext.data.utils import get_tokenizer
    from torchtext.vocab import build_vocab_from_iterator
except Exception as e:
    raise RuntimeError("torchtext no está disponible en este entorno") from e

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

specials = ['<unk>', '<pad>', '<bos>', '<eos>']
vocab = build_vocab_from_iterator(yield_tokens(ds), specials=specials, special_first=True)
vocab.set_default_index(vocab['<unk>'])

UNK_IDX = vocab['<unk>']
PAD_IDX = vocab['<pad>']
BOS_IDX = vocab['<bos>']
EOS_IDX = vocab['<eos>']

def text_to_ids(text: str):
    tokens = tokenizer(text)
    return [BOS_IDX] + [vocab[token] for token in tokens] + [EOS_IDX], tokens

# Muestra dos ejemplos
for s in sentences[:2]:
    ids, toks = text_to_ids(s)
    print("Texto:", s)
    print("Tokens:", toks)
    print("IDs:", ids, "\n")

assert isinstance(PAD_IDX, int) and PAD_IDX >= 0


**Ejercicio 10 - `collate_fn` con `pad_sequence` (batch_first=True)**

**Objetivo:** homogeneizar longitudes del lote.  
**Entrega:** `shape` y lote *padded* (vista parcial) e identificar `PAD_IDX`.


In [ ]:
# TODO: implementa collate_fn con pad_sequence
from torch.nn.utils.rnn import pad_sequence

def encode(texts):
    out = []
    for t in texts:
        ids, _ = text_to_ids(t)
        out.append(torch.tensor(ids, dtype=torch.long))
    return out

def collate_fn_batch_first(batch):
    seqs = encode(batch)
    batch_padded = pad_sequence(seqs, batch_first=True, padding_value=PAD_IDX)
    return batch_padded

dl_bf = DataLoader(ds, batch_size=4, shuffle=False, collate_fn=collate_fn_batch_first)
batch = next(iter(dl_bf))
print("Shape (B, T):", tuple(batch.shape))
print("Batch (vista parcial):\n", batch[:2, :10])
print("PAD_IDX:", PAD_IDX)

assert batch.dim() == 2 and batch.shape[0] == 4


**Ejercicio 11 -`batch_first=False`**

**Objetivo:** comparar `(B, T)` vs `(T, B)` y anotar preferencia (1 línea).


In [ ]:
# TODO: implementa collate_fn con batch_first=False
def collate_fn_time_first(batch):
    seqs = encode(batch)
    batch_padded = pad_sequence(seqs, batch_first=False, padding_value=PAD_IDX)
    return batch_padded

dl_tf = DataLoader(ds, batch_size=4, shuffle=False, collate_fn=collate_fn_time_first)
batch_tf = next(iter(dl_tf))
print("Shape (T, B):", tuple(batch_tf.shape))
print("Vista parcial:\n", batch_tf[:10, :2])

assert batch_tf.shape[1] == 4


**Ejercicio 12-Multi30k: primer vistazo**

**Objetivo:** cargar el `split='train'` y ver un ejemplo (DE, EN).  
> **Nota:** en entornos sin internet, esta celda puede fallar. Comenta la carga y escribe una breve explicación si no puedes ejecutarla.


In [ ]:
# TODO: intenta cargar Multi30k (opcional según entorno)
try:
    from torchtext.datasets import Multi30k
    train_iter = Multi30k(split='train', language_pair=('de','en'))
    example = next(iter(train_iter))
    print("Ejemplo DE:", example[0][:120], "...")
    print("Ejemplo EN:", example[1][:120], "...")
except Exception as e:
    print("Multi30k no disponible en este entorno ->", type(e).__name__, str(e)[:200])


**Ejercicio 13- spaCy tokenizers por idioma**

**Objetivo:** tokenizar con mejor calidad (DE/EN).  
> **Nota:** requiere modelos spaCy (`de_core_news_sm`, `en_core_web_sm`). Si no puedes instalarlos, compara con `basic_english` y comenta diferencias.


In [ ]:
# TODO: tokenizar con spaCy si está disponible
try:
    import spacy
    try:
        nlp_de = spacy.load("de_core_news_sm")
        nlp_en = spacy.load("en_core_web_sm")
        def spacy_tok_de(text): return [t.text for t in nlp_de(text)]
        def spacy_tok_en(text): return [t.text for t in nlp_en(text)]
        print("spaCy OK — muestra rápida:")
        print("DE:", spacy_tok_de("Das ist ein kurzer Testsatz."))
        print("EN:", spacy_tok_en("This is a short test sentence."))
    except OSError as oe:
        print("Modelos spaCy no instalados:", oe)
except Exception as e:
    print("spaCy no disponible ->", type(e).__name__, str(e)[:200])


**Ejercicio 14-Vocabularios por idioma + BOS/EOS**

**Objetivo:** construir `vocab_transform` y aplicar `tensor_transform` para incluir BOS/EOS.  
**Entrega:** un ejemplo con índices que incluya BOS/EOS.


In [ ]:
# TODO: arma vocabs por idioma y transforma a tensores con BOS/EOS
# Sugerencia: puedes reutilizar 'vocab' anterior para un solo idioma como ejemplo mínimo
def tensor_transform_with_bos_eos(text: str):
    ids, toks = text_to_ids(text)
    return torch.tensor(ids, dtype=torch.long)

ex = tensor_transform_with_bos_eos("Transformers no requieren flip en src.")
print("Ejemplo IDs (con BOS/EOS):", ex[:15], "...")
assert ex[0].item() == BOS_IDX


**Ejercicio 15 -`collate_fn` (pares) + DataLoader train/validate**

**Objetivo:** empaquetar `(src, tgt)` con *padding* y verificar *shapes*.  
**Recomendación:** simula pares `(src, tgt)` con oraciones ES->EN si no tienes dataset real.  
**Entrega:** `src.shape`, `tgt.shape` y 1 lote truncado.


In [ ]:
# TODO: simula pares (src, tgt) y arma un collate de traducción
pairs = [
    ("Hola desde Lima", "Hello from Lima"),
    ("El NLP requiere buenos datos", "NLP requires good data"),
    ("El padding alinea tensores", "Padding aligns tensors"),
    ("HF integra tokenizadores", "HF integrates tokenizers"),
    ("DataLoader crea lotes", "DataLoader creates batches"),
    ("spacy mejora la tokenización", "spacy improves tokenization"),
    ("Bucketing reduce padding waste", "Bucketing reduces padding waste"),
    ("Multi30k contiene pares", "Multi30k contains pairs")
]

from torch.utils.data import Dataset

class PairDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self): return len(self.pairs)
    def __getitem__(self, idx): return self.pairs[idx]

def encode_pair(src_t, tgt_t):
    src_ids, _ = text_to_ids(src_t)
    tgt_ids, _ = text_to_ids(tgt_t)
    return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

def collate_fn_translation(batch):
    src_list, tgt_list = [], []
    for src_t, tgt_t in batch:
        s, t = encode_pair(src_t, tgt_t)
        src_list.append(s); tgt_list.append(t)
    src_batch = pad_sequence(src_list, batch_first=True, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_list, batch_first=True, padding_value=PAD_IDX)
    return src_batch, tgt_batch

pair_ds = PairDataset(pairs)
pair_dl = DataLoader(pair_ds, batch_size=4, shuffle=False, collate_fn=collate_fn_translation)
src_b, tgt_b = next(iter(pair_dl))
print("src.shape:", tuple(src_b.shape), "tgt.shape:", tuple(tgt_b.shape))
print("src (vista parcial):\n", src_b[:2, :12])
print("tgt (vista parcial):\n", tgt_b[:2, :12])

assert src_b.shape[0] == tgt_b.shape[0] == 4


#### **Parte C-Retos**

> En esta sección solo se entrega el andamiaje (sin soluciones). Completa según tu ritmo.


**Ejercicio 16 - *Bucketing Sampler* por longitud**

**Objetivo:** reducir *padding waste*. Completa las funciones indicadas.


In [ ]:
# TODO: implementa bucketing simple por longitud
from typing import Iterable, List, Tuple

def sequence_length(x: torch.Tensor) -> int:
    # Longitud efectiva (sin PAD) si ya está *padded*; si no, usa len
    if isinstance(x, torch.Tensor):
        # cuenta tokens distintos a PAD
        return int((x != PAD_IDX).sum().item())
    return len(x)

def make_bins(indices: List[int], lengths: List[int], bin_size: int = 10) -> List[List[int]]:
    # TODO: agrupa indices por rangos de longitud (0-9, 10-19, ...)
    bins = {}
    for idx, L in zip(indices, lengths):
        b = (L // bin_size) * bin_size
        bins.setdefault(b, []).append(idx)
    return [bins[k] for k in sorted(bins.keys())]

def bucket_sampler(dataset: Dataset, batch_size: int = 4, bin_size: int = 10) -> Iterable[List[int]]:
    # TODO: devuelve listas de índices por lote, agrupadas por bins
    indices = list(range(len(dataset)))
    # nota: aquí medimos longitud de texto (aprox) antes de tokenizar, a modo de demo
    lengths = [len(dataset[i][0]) if isinstance(dataset[i], tuple) else len(dataset[i]) for i in indices]
    buckets = make_bins(indices, lengths, bin_size=bin_size)
    for bucket in buckets:
        # yieldea en trozos del tamaño batch
        for i in range(0, len(bucket), batch_size):
            yield bucket[i:i+batch_size]

print("Base lista para bucketing. Completa mediciones y compara radio de padding.")


**Ejercicio 17 - Métrica de *padding waste* + logging por lote**

**Objetivo:** instrumentar medición continua.  

**Tareas:**
1) En cada batch, calcula `pad_count` y `token_count`.  
2) Acumula por época y registra p50/p90/p99 del *waste*.  



In [ ]:
## Tu respuesta

**Ejercicio 18 - Máscaras de atención**

**Objetivo:** derivar `attention_mask` y máscara causal del *decoder*.


In [ ]:
# TODO: a partir de src_b / tgt_b, genera masks
def make_attention_mask(batch_padded: torch.Tensor, pad_idx: int) -> torch.Tensor:
    # 1 = token válido, 0 = PAD
    return (batch_padded != pad_idx).to(torch.long)

def make_causal_mask(T: int) -> torch.Tensor:
    # Triangular superior en 0; permite ver solo pasado/presente
    return torch.tril(torch.ones((T, T), dtype=torch.long))

# Ejemplo con los lotes previos (si existen)
try:
    src_mask = make_attention_mask(src_b, PAD_IDX)
    tgt_mask = make_attention_mask(tgt_b, PAD_IDX)
    causal = make_causal_mask(tgt_b.shape[1])
    print("src_mask:", tuple(src_mask.shape), "tgt_mask:", tuple(tgt_mask.shape), "causal:", tuple(causal.shape))
except NameError:
    print("Ejecuta antes la celda del Ejercicio 15 para disponer de src_b/tgt_b.")


**Ejercicio 19 - *Loss* ignorando PAD + *teacher forcing* (bucle mínimo)**

**Objetivo:** preparar un entrenamiento minimalista.  
**Tareas:**
1) Implementa una función de *loss* que ignore `PAD_IDX` (usar `ignore_index`).  
2) Arma un bucle simulado de entrenamiento con *teacher forcing* (no requiere modelo complejo,  puede ser *dummy* que solo pruebe la forma).  


**Ejercicio 20 - Sustituir TorchText por *tokenizador HF* (compatibilidad de checkpoint)**

**Objetivo:** usar el *tokenizer* del modelo (p.ej., T5/mT5) para garantizar compatibilidad.  
**Tareas:**
1) Reemplaza `token_transform`/`vocab_transform` por `AutoTokenizer` del checkpoint.  
2) Asegura `padding=True`, `truncation=True`, `return_tensors='pt'` y crea una `collate_fn` análoga.  


**Ejercicio 21 - Portar a `datasets` (HF) + `map` + `DataCollatorWithPadding`**

**Objetivo:** comparar pipelines.  
**Tareas:**
1) Carga un *split* compatible (o usa un dataset de texto de HF).  
2) Aplica `map` con el tokenizador y usa `DataCollatorWithPadding`.  
3) Compara tiempos/ergonomía vs TorchText.  


**Ejercicio 22 - *Tests* mínimos (pytest) para el *pipeline* de datos**

**Objetivo:** validar contratos.  
**Tareas:** crea *tests* que verifiquen:  
- (a) presencia de BOS/EOS,  
- (b) `PAD_IDX` solo en cola del *padding*,  
- (c) *shapes* esperados y `dtype=int64`,  
- (d) reversa aplicada solo a `src`.  


In [ ]:
## Tus respuestas

#### **Entrega**
- Evidencias mínimas por ejercicio (shapes, fragmentos de tensores/tablas).  
- Comentarios breves explicando decisiones.  
- Sin credenciales ni datos sensibles.  
- `README.md` con qué funcionó y qué no (5-10 líneas).  
- Roles documentados si trabajaste en equipo.  


